## Import data

In [1]:
import sys
sys.path.append("../.")

In [39]:
import pathlib
import numpy as np
from math import pi
import matplotlib.pyplot as plt
import torch
from lips.benchmark.powergridBenchmark import PowerGridBenchmark
from gnn_powergrid.dataset import prepare_dataset

In [3]:
env_name = "l2rpn_case14_sandbox"

path = pathlib.Path().resolve().parent
BENCH_CONFIG_PATH = path / "configs" / (env_name + ".ini")
DATA_PATH = path / "Datasets" / env_name / "DC"
LOG_PATH = path / "lips_logs.log"

benchmark = PowerGridBenchmark(benchmark_path=DATA_PATH,
                               benchmark_name="Benchmark4",#"DoNothing",
                               load_data_set=True,
                               config_path=BENCH_CONFIG_PATH,
                               log_path=LOG_PATH)

In [4]:
print(benchmark.train_dataset.size)
print(benchmark._test_dataset.size)

100000
10000


In [5]:
device = torch.device("cpu") # or "cuda:0" if you have any GPU
train_loader, val_loader, test_loader, test_ood_loader = prepare_dataset(benchmark=benchmark, 
                                                                         batch_size=128, 
                                                                         device=device)
batch = next(iter(test_loader))
print(batch)

*******Train dataset*******
Train data size: 100000


  0%|          | 0/4 [00:00<?, ?it/s]

*******Validation dataset*******
Validation data size: 10000


  0%|          | 0/4 [00:00<?, ?it/s]

*******Test dataset*******
Test data size : 10000


  0%|          | 0/4 [00:00<?, ?it/s]

*******OOD dataset*******
OOD data size: 10000


  0%|          | 0/4 [00:00<?, ?it/s]

DataBatch(x=[3584, 2], edge_index=[2, 6656], edge_attr=[6656], y=[3584, 1], edge_index_no_diag=[2, 4864], edge_attr_no_diag=[4864], ybus=[3584, 28], batch=[3584], ptr=[129])


In [7]:
batch.ybus.dtype

torch.float32

## define the model

#### Layers

In [57]:
import torch
from torch_geometric.nn import MessagePassing

class GPGinput(MessagePassing):
    def __init__(self,
                 ref_node,
                 device="cpu"
                 ):
        super().__init__(aggr="add")
        self.ref_node = ref_node
        self.device = device

    def forward(self, init_theta, batch):

        aggr_msg = self.propagate(batch.edge_index_no_diag,
                                  y=init_theta,
                                  edge_weights=batch.edge_attr_no_diag * 100.0
                                 )
        
        # keep only the diagonal elements of the ybus 3D tensors
        n_bus = batch.ybus.size()[1]
        n_sub = n_bus / 2
        ybus = batch.ybus.view(-1, n_bus, n_bus) * 100.0
        ybus = ybus * torch.eye(*ybus.shape[-2:], device=self.device).repeat(ybus.shape[0], 1, 1)
        denominator = torch.hstack([ybus[i].diag() for i in range(len(ybus))]).reshape(-1,1)
        
        input_node_power = (batch.x[:,0] - batch.x[:,1]).view(-1,1)
        numerator = input_node_power - aggr_msg
        
        indices = torch.where(denominator.flatten()!=0.)[0]
        out = torch.zeros_like(denominator)
        out[indices] = torch.divide(numerator[indices], denominator[indices])

        #we impose that node 0 has theta=0
        out = out.view(-1, n_bus, 1) - out.view(-1,n_bus,1)[:,self.ref_node].repeat_interleave(n_bus, 1).view(-1, n_bus, 1)
        out = out.flatten().view(-1,1)
        
        # imposing theta=0 for the bus which are not used
        out[denominator==0] = 0
        
        return out, aggr_msg
    
    def message(self, y_j, edge_weights):
        tmp = y_j * edge_weights.view(-1,1)
        return tmp
    
    def update(self, aggr_out):
        return aggr_out
    
class GPGintermediate(MessagePassing):
    def __init__(self,
                 ref_node,
                 device="cpu"):
        super().__init__(aggr="add")
        self.theta = None
        self.ref_node = ref_node
        self.device = device
        
    
    def forward(self, batch, theta):
        self.theta = theta
        
        aggr_msg = self.propagate(batch.edge_index_no_diag,
                                  y=self.theta,
                                  edge_weights=batch.edge_attr_no_diag * 100.0
                                 )

        n_bus = batch.ybus.size()[1]
        # keep only the diagonal elements of the ybus 3D tensors for denominator part
        ybus = batch.ybus.view(-1, n_bus, n_bus) * 100.0
        # ybus = ybus * torch.eye(*ybus.shape[-2:], device=self.device).repeat(ybus.shape[0], 1, 1)
        # denominator = ybus[ybus.nonzero(as_tuple=True)].view(-1,1)
        denominator = torch.hstack([ybus[i].diag() for i in range(len(ybus))]).reshape(-1,1)
        
        input_node_power = (batch.x[:,0] - batch.x[:,1]).view(-1,1)
        numerator = input_node_power - aggr_msg
        # out = (input_node_power - aggr_msg) / denominator
        indices = torch.where(denominator.flatten()!=0.)[0]
        out = torch.zeros_like(denominator)
        out[indices] = torch.divide(numerator[indices], denominator[indices])

        #we impose that reference node has theta=0
        out = out.view(-1, n_bus, 1) - out.view(-1,n_bus,1)[:,self.ref_node].repeat_interleave(n_bus, 1).view(-1, n_bus, 1)
        out = out.flatten().view(-1,1)
        #we impose the not used buses to have theta=0
        out[denominator==0] = 0
        
        return out, aggr_msg

    def message(self, y_i, y_j, edge_weights):
        tmp = y_j * edge_weights.view(-1,1)
        return tmp

    def update(self, aggr_out):
        return aggr_out

class LocalConservationLayer(MessagePassing):
    """
    Same as power equilibrium equations
    
    It can be also injected as inputs to the next layer
    """
    def __init__(self):
        super().__init__(aggr="add")
        self.thetas = None
        
    def forward(self, batch, thetas=None):
        self.thetas = thetas

        aggr_message = self.propagate(batch.edge_index,
                                      y=self.thetas,
                                      edge_weights=batch.edge_attr * 100)

        input_node_power = (batch.x[:,0] - batch.x[:,1]).view(-1,1)
        nodal_error = input_node_power - aggr_message

        return nodal_error

    def message(self, y_i, y_j, edge_weights):
        tmp = y_j * edge_weights.view(-1,1)
        return tmp

#### Model

In [77]:
from torch.nn import Linear
from torch.nn import Module

class GPGmodel(Module):
    def __init__(self,
                 ref_node,
                 input_dim=2,
                 output_dim=1,
                 embedding_size=16,
                 num_gnn_layers=10,
                 device="cpu"):
        super().__init__()
        self.ref_node = ref_node
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.embedding_size = embedding_size
        self.num_gnn_layers = num_gnn_layers
        
        self.device = device

        self.embedding_layer = None
        self.input_layer = None
        self.lc_layer = None
        self.inter_layers = None
        self.decoding_layer = None

        self.build_model()

    def build_model(self):
        self.embedding_layer = Linear(self.input_dim, self.output_dim)
        self.input_layer = GPGinput(ref_node=self.ref_node, device=self.device)
        self.lc_layer = LocalConservationLayer()
        self.inter_layer = GPGintermediate(ref_node=self.ref_node, device=self.device)
        #self.inter_layers = ModuleList([GPGintermediate(device=self.device) for _ in range(self.num_gnn_layers)])
        self.decoding_layer = Linear(self.output_dim, self.output_dim)

    def forward(self, batch):
        errors = []
        init_theta = self.embedding_layer(batch.x) # start by NN init
        # init_theta = torch.zeros_like(batch.y, dtype=batch.y.dtype) # flat start 
        out, _ = self.input_layer(init_theta, batch)
        nodal_error = self.lc_layer(batch, out)
        errors.append(abs(nodal_error).sum())
        
        #for gnn_layer, lc_layer_ in zip(self.inter_layers, itertools.repeat(self.lc_layer)):
        for _ in range(self.num_gnn_layers):
            out, _ = self.inter_layer(batch, out)
            nodal_error = self.lc_layer(batch, out)
            errors.append(abs(nodal_error).sum())

        # out = self.decoding_layer(out)
        return out, errors

##### Try input layer and local conservation error

In [17]:
init_theta = torch.zeros_like(batch.y, dtype=batch.y.dtype)
input_layer = GPGinput(ref_node=0, device="cpu")
lc_layer = LocalConservationLayer()

In [18]:
out, msg = input_layer(init_theta=init_theta, batch=batch)
nodal_error = lc_layer(batch, out) 

In [108]:
nodal_errors_batch = torch.stack(torch.chunk(abs(nodal_error.flatten()), 128))
torch.sum(nodal_errors_batch.sum(dim=1)/14)/128

tensor(10.7121, dtype=torch.float64)

In [56]:
# same as above
(abs(nodal_error).sum() / 14)/128

tensor(10.7121, dtype=torch.float64)

##### Try the model

In [27]:
gpg_model = GPGmodel(ref_node=0)
gpg_model.to(device)
gpg_model.to(batch.x.dtype)
out, errors = gpg_model(batch)

In [35]:
len(batch)

128

#### Train and predict

In [78]:
from tqdm.auto import tqdm

def train_model(model,
                train_loader,
                optimizer,
                val_loader=None,
                epochs=10):
    train_losses = list()
    val_losses = list()
    # loss_func = MSELoss()

    for epoch in tqdm(range(epochs)):
        total_loss = 0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            pred, error = model(batch)
            loss = error[-1]
            loss.backward()
            optimizer.step()
            #total_loss += (loss * len(batch.x))
            total_loss += loss / 14
            #total_loss /= len(batch)
        total_loss = total_loss.item() / len(train_loader.dataset)

        if val_loader is not None:
            _, _, val_loss = eval_model(model, val_loader)
            val_losses.append(val_loss)
            print("Epoch {}. Loss: {:.4f}. val_theta loss: {:.4f}".format(
                epoch, total_loss, val_loss))
        else:
            print("Epoch {}. Loss: {:.4f}".format(epoch, total_loss))
        train_losses.append(total_loss)        

    return model, train_losses, val_losses

def eval_model(model, loader):
    model.eval()
    predictions = list()
    observations = list()
    total_loss = 0
    #total_constraint = 0
    # loss_func = MSELoss()
    with torch.no_grad():
        for batch in loader:
            data = batch.x
            #pred, active_powers_bus = model(batch)
            pred, errors = model(batch)
            true = batch.y
            # loss computed on local conservation
            #constraint = loss_func(message, power_at_node.view(-1,1))
            # loss = (power_at_node - active_powers_bus).norm(2)
            # loss_2 = loss_func(pred, true)
            # loss = loss_1 + loss_2
            # loss = loss_func(pred, true)
            loss = errors[-1]
            #total_loss += loss * len(data)
            total_loss += loss / 14
            #total_constraint += constraint * len(data)
            predictions.append(pred)
            observations.append(true)
    predictions = torch.vstack(predictions)
    observations = torch.vstack(observations)
    mean_loss = total_loss.item() / len(loader.dataset)
    #total_constraint = total_constraint.item() / len(loader.dataset)
    
    return predictions, observations, mean_loss

In [79]:
gpg_model = GPGmodel(ref_node=0, num_gnn_layers=50, device=device)
gpg_model.to(device)
optimizer = torch.optim.Adam(gpg_model.parameters(), lr=6e-4)

In [80]:
model, train_losses, val_losses = train_model(gpg_model, 
                                              train_loader, 
                                              optimizer, 
                                              val_loader, 
                                              epochs=10)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0. Loss: 9.5998. val_theta loss: 5.1224
Epoch 1. Loss: 0.7958. val_theta loss: 0.0524
Epoch 2. Loss: 0.0449. val_theta loss: 0.0519
Epoch 3. Loss: 0.0450. val_theta loss: 0.0500
Epoch 4. Loss: 0.0450. val_theta loss: 0.0503
Epoch 5. Loss: 0.0451. val_theta loss: 0.0505
Epoch 6. Loss: 0.0451. val_theta loss: 0.0507
Epoch 7. Loss: 0.0451. val_theta loss: 0.0507
Epoch 8. Loss: 0.0451. val_theta loss: 0.0507
Epoch 9. Loss: 0.0450. val_theta loss: 0.0516


In [81]:
predictions, observations, mean_loss = eval_model(gpg_model, test_loader)

In [82]:
observations

tensor([[ 0.0000],
        [-1.3042],
        [-4.4101],
        ...,
        [ 0.0000],
        [ 0.0000],
        [ 0.0000]])

In [83]:
predictions * (180/pi)

tensor([[ 0.0000],
        [-1.3040],
        [-4.4094],
        ...,
        [ 0.0000],
        [ 0.0000],
        [ 0.0000]])

In [84]:
MAPE = abs((observations - (predictions * (180/pi))) / (observations + 1e-5)).mean()
print(MAPE)

tensor(0.0011)


In [85]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
print("MAE: ", mean_absolute_error((predictions*(180/pi)).cpu().numpy(), observations.cpu().numpy()))
print("MAPE: ", mean_absolute_percentage_error((predictions*(180/pi)).cpu().numpy(), observations.cpu().numpy()))

MAE:  0.012845651
MAPE:  0.0011300456


In [86]:
from gnn_powergrid.dataset.utils.graph_utils import get_all_active_powers
from gnn_powergrid.dataset.utils.solver_utils import get_obs


env, obs = get_obs(benchmark)

my_predictions = {}
p_ors_pred, p_exs_pred = get_all_active_powers(benchmark._test_dataset.data, 
                                               obs, 
                                               theta_bus=(predictions*(180/pi)).view(-1, obs.n_sub*2).cpu())
my_predictions["p_or"] = p_ors_pred
my_predictions["p_ex"] = p_exs_pred

  0%|          | 0/10000 [00:00<?, ?it/s]

In [87]:
my_predictions

{'p_or': array([[ 3.8463341e+01,  3.1221951e+01,  2.7378098e+01, ...,
          5.2651076e+00,  2.6037907e-02, -2.5557652e+01],
        [ 3.7183605e+01,  3.1110968e+01,  2.6624983e+01, ...,
          8.4811983e+00,  6.0061453e-04, -2.3301374e+01],
        [ 3.7648624e+01,  3.1921959e+01,  2.6158779e+01, ...,
          1.7716627e+01,  4.3142732e-04, -2.7856489e+01],
        ...,
        [ 3.9061081e+01,  3.0917017e+01,  3.0640781e+01, ...,
          1.2027138e+01, -1.3806812e+01, -3.3561333e+01],
        [ 2.8853081e+01,  4.0230785e+01,  3.4583946e+01, ...,
          8.5671606e+00, -1.2303131e+01, -2.7189844e+01],
        [ 3.8149796e+01,  3.0641281e+01,  2.9780848e+01, ...,
          6.3635674e+00, -1.1396524e+01, -2.8885117e+01]], dtype=float32),
 'p_ex': array([[-3.8463341e+01, -3.1221951e+01, -2.7378098e+01, ...,
         -5.2651076e+00, -2.6037907e-02,  2.5557652e+01],
        [-3.7183605e+01, -3.1110968e+01, -2.6624983e+01, ...,
         -8.4811983e+00, -6.0061453e-04,  2.3301374e

In [88]:
from lips.evaluation.powergrid_evaluation import PowerGridEvaluation


evaluation = PowerGridEvaluation.from_benchmark(benchmark)
metrics = evaluation.evaluate(observations=benchmark._test_dataset.data, 
                              predictions=my_predictions, 
                              env=env)

In [89]:
metrics

{'ML': {'MSE_avg': {'p_or': 0.043757639825344086,
   'p_ex': 0.043757639825344086},
  'MAE_avg': {'p_or': 0.054794203490018845, 'p_ex': 0.054794203490018845},
  'MAPE_avg': {'p_or': 2163348340736.0, 'p_ex': 2163348340736.0},
  'MAPE_90_avg': {'p_or': 0.008111423981247319, 'p_ex': 0.008111423981247319},
  'MAPE_10_avg': {'p_or': 0.004253085751504593, 'p_ex': 0.004253085751504593}},
 'Physics': {'LOSS_POS': {'violation_proportion': 0.0},
  'DISC_LINES': {'p_or': 0.0, 'p_ex': 0.0, 'violation_proportion': 0.0},
  'CHECK_LOSS': {'violation_percentage': 0.0},
  'CHECK_GC': {'violation_percentage': 0.0,
   'mae': 1.0942078e-05,
   'wmape': 1.0},
  'CHECK_LC': {'violation_percentage': 57.28785714285715,
   'mae': 0.05239958356437928,
   'mape': 0.0025080187147790292}}}